<a href="https://colab.research.google.com/github/abel-bernabeu/autoencoder/blob/master/compressor_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description

This notebook is an implementation of the enconder/decoder deep learning architecture from "Lossy image compression with compression autoencoders", by 
Lucas Theis, Wenzhe Shi, Andrew Cunningham & Ferenc Husz, published in 2017
(https://arxiv.org/pdf/1703.00395v1.pdf).

# Setup

In [ ]:
# Get the dataset if needed
import os.path
if not os.path.isdir('./data'):
    !rm  -rf image_dataset_part-a.zip
    !mkdir data -p
    !cd data ; wget https://www.dropbox.com/s/91rpg5dqkhhhkzu/image_dataset_part-a.zip
    !cd data ; unzip -q image_dataset_part-a.zip && rm image_dataset_part-a.zip 

# Get the latest source code if needed
if not os.path.isdir('./autoencoder'):
    !wget https://www.dropbox.com/s/jfu0ksttohnklkq/autoencoder-master.zip && \
    unzip -q autoencoder-master.zip && rm autoencoder-master.zip \
    mv autoencoder-master/autoencoder/ . && \
    rm autoencoder-master -rf

if not os.path.isdir('./share'):
    try:
        # Try to mount share from Google Drive when on Collab
        from google.colab import driveX
        drive.mount('/content/drive/')
        !ln -s  /content/drive/My\ Drive/archive/2020/aidl/ share
    except:
        # The fallback for when not in Collab is to download share from Dropbox
        !wget https://www.dropbox.com/s/76w9gsga8mz5ve4/share.tgz && tar xzf share.tgz && rm share.tgz

In [ ]:
import autoencoder.models
import autoencoder.utils

# Sparsity at 1/2

Our baseline model effort focuses on training the proposed neural network with the maximum possible accuracy, but not investing any effort in quantization or entropic coding of the features.

This model only achieves a 50% compression when the input is in Float32 but it is used for three purposes:

- prove that the input can be reconstructed accurately with the kind of neural network that is proposed in the paper.

- set an upper bound on accuracy (**43 db**)

- give an estimation of how long it takes to train a state of the art compression model (**4 days on a Tesla P100**)


## Hyperparameters

In [ ]:
hparams = {
    'batch_size' : 32,
    'lr' : 1e-3,
    'device' : 'cuda',
    'block_width' : 128,
    'block_height' : 128,
    'hidden_state_num_channels' : 96,
    'train_dataset_size' : 5000,
    'test_dataset_size' : 500,
    'num_epochs' : 12577,
    'num_workers' : 4,
    'name' : "twitter-compressor",
    'port' : 6100,
    'checkpointing_freq' : 10,
    'inference_freq' : 20,
}

!mkdir -p share/{hparams['name']}

## Model

In [ ]:
model = autoencoder.models.TwitterCompressor(hidden_state_num_channels = hparams['hidden_state_num_channels'])

## Dataset

In [ ]:
train_loader, test_loader, few_train_x, few_train_y, few_test_x, few_test_y = autoencoder.utils.create_dataloaders(hparams)

## TensorBoard

In [ ]:
try:
    # When on Google Colab try to launch an embedded TensorBoard
    from google.colab import drive
    %load_ext tensorboard
    from tensorboard import notebook
    notebook.start('--logdir share/' + hparams['name'] + '/runs/ --port ' + str(hparams['port']))
except:
    pass

## Training

In [ ]:
try:
  autoencoder.utils.train(hparams=hparams, \
        model=model, \
        train_loader=train_loader, \
        test_loader=test_loader, \
        few_train_x=few_train_x, few_train_y=few_train_y, \
        few_test_x=few_test_x, few_test_y=few_test_y)
except KeyboardInterrupt:
    print('Exiting from training early')

# Sparsity at 1/4

## Hyperparameters

In [ ]:
hparams = {
    'batch_size' : 40,
    'lr' : 1e-6,
    'device' : 'cuda',
    'block_width' : 224,
    'block_height' : 224,
    'hidden_state_num_channels' : 48,
    'train_dataset_size' : 1000,
    'test_dataset_size' : 500,
    'num_epochs' : 16000,
    'num_workers' : 4,
    'name' : "sparse-twitter-compressor",
    'port' : 6200,
    'checkpointing_freq' : 10,
    'inference_freq' : 20,
}

!mkdir -p share/{hparams['name']}

## Model

In [ ]:
model = autoencoder.models.TwitterCompressor(hidden_state_num_channels = hparams['hidden_state_num_channels'])

## Dataset

In [ ]:
train_loader, test_loader, few_train_x, few_train_y, few_test_x, few_test_y = autoencoder.utils.create_dataloaders(hparams)

## TensorBoard

In [ ]:
try:
    # When on Google Colab try to launch an embedded TensorBoard
    from google.colab import drive
    %load_ext tensorboard
    from tensorboard import notebook
    notebook.start('--logdir share/' + hparams['name'] + '/runs/ --port ' + str(hparams['port']))
except:
    pass

## Training

In [ ]:
try:
  autoencoder.utils.train(hparams=hparams, \
        model=model, \
        train_loader=train_loader, \
        test_loader=test_loader, \
        few_train_x=few_train_x, few_train_y=few_train_y, \
        few_test_x=few_test_x, few_test_y=few_test_y)
except KeyboardInterrupt:
    print('Exiting from training early')

# Uniform quantization to 3 bits

## Hyperparameters

In [ ]:
hparams = {
    'batch_size' : 40,
    'lr' : 1e-6,
    'device' : 'cuda',
    'block_width' : 224,
    'block_height' : 224,
    'hidden_state_num_channels' : 48,
    'train_dataset_size' : 1000,
    'test_dataset_size' : 500,
    'num_epochs' : 2500,
    'num_workers' : 4,
    'name' : "uniform-quant-twitter-compressor",
    'port' : 6300,
    'checkpointing_freq' : 10,
    'inference_freq' : 20,
}

!mkdir -p  share/{hparams['name']}

## Model

In [ ]:
qmodel = autoencoder.models.QuantizingCompressionAutoencoder(num_bits=3)

# Transfer learning from the non-quantized model
qmodel.encoder = model.encoder
qmodel.decoder = model.decoder

# Freeze the encoder
for param in qmodel.encoder.parameters():
    param.requires_grad = False

## Dataset

In [ ]:
train_loader, test_loader, few_train_x, few_train_y, few_test_x, few_test_y = autoencoder.utils.create_dataloaders(hparams)

## TensorBoard

In [ ]:
try:
    # When on Google Colab try to launch an embedded TensorBoard
    from google.colab import drive
    %load_ext tensorboard
    from tensorboard import notebook
    notebook.start('--logdir share/' + hparams['name'] + '/runs/ --port ' + str(hparams['port']))
except:
    pass

## Training

In [ ]:
try:
  autoencoder.utils.train(hparams=hparams, \
        model=qmodel, \
        train_loader=train_loader, \
        test_loader=test_loader, \
        few_train_x=few_train_x, few_train_y=few_train_y, \
        few_test_x=few_test_x, few_test_y=few_test_y)
except KeyboardInterrupt:
    print('Exiting from training early')

# Sparsity at 1/8

## Hyperparameters

In [ ]:
hparams = {
    'batch_size' : 40,
    'lr' : 1e-6,
    'device' : 'cuda',
    'block_width' : 224,
    'block_height' : 224,
    'hidden_state_num_channels' : 24,
    'train_dataset_size' : 1000,
    'test_dataset_size' : 500,
    'num_epochs' : 110000,
    'num_workers' : 4,
    'name' : "sparse-2-twitter-compressor",
    'port' : 6400,
    'checkpointing_freq' : 10,
    'inference_freq' : 20,
}

!mkdir -p share/{hparams['name']}

## Model

In [ ]:
model = autoencoder.models.TwitterCompressor(hidden_state_num_channels = hparams['hidden_state_num_channels'])

## Dataset

In [ ]:
train_loader, test_loader, few_train_x, few_train_y, few_test_x, few_test_y = autoencoder.utils.create_dataloaders(hparams)

## TensorBoard

In [ ]:
try:
    # When on Google Colab try to launch an embedded TensorBoard
    from google.colab import drive
    %load_ext tensorboard
    from tensorboard import notebook
    notebook.start('--logdir share/' + hparams['name'] + '/runs/ --port ' + str(hparams['port']))
except:
    pass

## Training

In [ ]:
try:
  autoencoder.utils.train(hparams=hparams, \
        model=model, \
        train_loader=train_loader, \
        test_loader=test_loader, \
        few_train_x=few_train_x, few_train_y=few_train_y, \
        few_test_x=few_test_x, few_test_y=few_test_y)
except KeyboardInterrupt:
    print('Exiting from training early')

# Uniform quantization to 6 bits

## Hyperparameters

In [ ]:
hparams = {
    'batch_size' : 40,
    'lr' : 1e-8,
    'device' : 'cuda',
    'block_width' : 224,
    'block_height' : 224,
    'hidden_state_num_channels' : 24,
    'train_dataset_size' : 1000,
    'test_dataset_size' : 500,
    'num_epochs' : 20000,
    'num_workers' : 4,
    'name' : "uniform-quant-2-twitter-compressor",
    'port' : 6500,
    'checkpointing_freq' : 10,
    'inference_freq' : 20,
}

!mkdir -p share/{hparams['name']}

## Model

In [ ]:
qmodel = autoencoder.models.QuantizingCompressionAutoencoder(num_bits=6)

# Transfer learning from the non-quantized model
qmodel.encoder = model.encoder
qmodel.decoder = model.decoder

# Freeze the encoder
for param in qmodel.encoder.parameters():
    param.requires_grad = False

## Dataset

In [ ]:
train_loader, test_loader, few_train_x, few_train_y, few_test_x, few_test_y = autoencoder.utils.create_dataloaders(hparams)

## TensorBoard

In [ ]:
try:
    # When on Google Colab try to launch an embedded TensorBoard
    from google.colab import drive
    %load_ext tensorboard
    from tensorboard import notebook
    notebook.start('--logdir share/' + hparams['name'] + '/runs/ --port ' + str(hparams['port']))
except:
    pass

## Training

In [ ]:
try:
  autoencoder.utils.train(hparams=hparams, \
        model=qmodel, \
        train_loader=train_loader, \
        test_loader=test_loader, \
        few_train_x=few_train_x, few_train_y=few_train_y, \
        few_test_x=few_test_x, few_test_y=few_test_y)
except KeyboardInterrupt:
    print('Exiting from training early')